In [2]:
import os
import subprocess


os.environ["COMSPEC"] = "powershell"

result = subprocess.run(
    ["Write-Host", "Hello from the child"],
    capture_output=True,
    encoding="utf-8",
    shell=True)

result.check_returncode() # No exception means clean exit
print(result.stdout)

Hello from the child



In [3]:
subprocess.run(["ls", "-l", './'], capture_output=True)

CompletedProcess(args=['ls', '-l', './'], returncode=0, stdout=b'total 975\n-rw-r--r-- 1 LT04 197121   1086 Apr 27 14:40 Item_1python_version.ipynb\n-rw-r--r-- 1 LT04 197121     40 Apr 25 15:05 README.md\n-rw-r--r-- 1 LT04 197121    880 Apr 27 14:40 Untitled.ipynb\ndrwxr-xr-x 1 LT04 197121      0 Apr 27 14:40 __pycache__\n-rw-r--r-- 1 LT04 197121   3573 Apr 27 14:40 basic_class.ipynb\n-rw-r--r-- 1 LT04 197121   5003 Apr 29 08:22 basic_config.py\n-rw-r--r-- 1 LT04 197121  12429 Apr 27 14:40 basic_dict.ipynb\n-rw-r--r-- 1 LT04 197121   7413 Apr 27 14:40 basic_filesystem.ipynb\n-rw-r--r-- 1 LT04 197121   3076 Apr 27 14:40 basic_inheritance.ipynb\n-rw-r--r-- 1 LT04 197121   5387 Apr 27 14:40 basic_recursive.ipynb\n-rw-r--r-- 1 LT04 197121     24 Apr 27 14:40 bytes.bin\n-rw-r--r-- 1 LT04 197121    776 Apr 29 08:22 config.py\n-rw-r--r-- 1 LT04 197121     32 Apr 27 14:40 dictionary.json\ndrwxr-xr-x 1 LT04 197121      0 Apr 27 14:40 filesystem_test\n-rw-r--r-- 1 LT04 197121   1464 Apr 27 14:40

In [4]:
import time
#subprocess.Popen(["sleep", "1"]) with in put is a list of command line and it's arguments
proc = subprocess.Popen(["sleep", "1"])
#CreateProcess() class give us some other method to interact with with subprocess
#process.poll() return None if process is not finish if finished return status code
while proc.poll() is None:
    print("Working...")
    time.sleep(0.4)

print("Exit status", proc.poll())

Working...
Working...
Working...
Exit status 0


In [5]:
import time

start = time.time()
sleep_procs = []
for _ in range(10):
    #run 10 subprocess seperately
    proc = subprocess.Popen(["sleep", "1"])
    sleep_procs.append(proc)
    
for proc in sleep_procs:
    #communicate() method will return stdout, stderr
    stdout_data, stderr_data = proc.communicate()
    print('stdout_data: {}, stderr_data: {}'.format(stdout_data, stderr_data))

end = time.time()
delta = end - start

print(f"Finished in {delta:.3} seconds")

stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
stdout_data: None, stderr_data: None
Finished in 1.18 seconds


In [8]:
import os


def run_encrypt(data):
    #data is a python object
    env = os.environ.copy()
    env["password"] = "4(;QlJ?mVXv?^|+q@UmR%eQaq|Aqh):?"
    proc = subprocess.Popen(
        ["openssl", "enc", "-des3", "-pass", "env:password"],
        env=env,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE)
    #subprocess.PIPE will help you get data from python current program
    proc.stdin.write(data) #get data from python object
    proc.stdin.flush() # Ensure that the child gets input
    return proc

procs = []
for _ in range(3):
    data = os.urandom(10)

    proc = run_encrypt(data)
    procs.append(proc)

for proc in procs:
    out, _ = proc.communicate()
    print(out[-10:])

b'\x87\xb0~\xf0\x95\xd2\x00\x90\x86\xbd'
b'n@\xd07\x84\x92\x99\x91\xaeY'
b'\xccU\x15\x0b\xd62\xdc\xb6g\xf0'


In [9]:
def run_hash(input_stdin):
    # run_hash take input from other subprocess. input_stdin = other_subprocess.stdout
    return subprocess.Popen(
        ["openssl", "dgst", "-whirlpool", "-binary"],
        stdin=input_stdin,
        stdout=subprocess.PIPE)

encrypt_procs = []
hash_procs = []

for _ in range(3):
    data = os.urandom(100)
    encrypt_proc = run_encrypt(data)
    encrypt_procs.append(encrypt_proc)
    hash_proc = run_hash(encrypt_proc.stdout)
    hash_procs.append(hash_proc)
    # Ensure that the child consumes the input stream and 
    # the communicate() method doesn't inadvertently steal
    # input from the child. Also lets SIGPIPE propagate to 
    # upstream process if the downstream process dies.
    encrypt_proc.stdout.close()
    encrypt_proc.stdout = None

for proc in encrypt_procs:
    proc.communicate()
    assert proc.returncode == 0

for proc in hash_procs:
    out, _ = proc.communicate()
    print(out[-10:])
    assert proc.returncode == 0

b'\x07\xcd\x8b\x88;\xd0O\x88\xbe7'
b'\x19$\xc8\xf6\xd8\xd5\x9bn\xe3\x90'
b'\xb3\x14S[\xf5|q\x0c[p'


In [10]:
proc = subprocess.Popen(['sleep', '10'])

try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    #terminate and wait is use together, termigate and wait for terminate to complete
    proc.terminate()
    proc.wait()

#poll() check process terminate or not, and return return code. 0 means success, other means error
print('Exit status', proc.poll())

Exit status 1


In [25]:
import subprocess

subprocess.run(['ls'])

README.md
item_36_subprocess.ipynb
venv


CompletedProcess(args=['ls'], returncode=0)

In [30]:
process_one = subprocess.run(['ls', '-la'], capture_output=True, text=True) #run take input as a list.
print(process_one.stdout)

total 56
drwxr-xr-x 4 toanbui1991 toanbui1991  4096 Apr 25 14:50 .
drwxr-xr-x 3 toanbui1991 toanbui1991  4096 Apr 25 14:43 ..
drwxr-xr-x 8 toanbui1991 toanbui1991  4096 Apr 25 14:43 .git
-rw-r--r-- 1 toanbui1991 toanbui1991  1799 Apr 25 14:43 .gitignore
-rw-r--r-- 1 toanbui1991 toanbui1991    40 Apr 25 14:43 README.md
-rw-r--r-- 1 toanbui1991 toanbui1991 29986 Apr 25 16:48 item_36_subprocess.ipynb
drwxr-xr-x 6 toanbui1991 toanbui1991  4096 Apr 25 14:50 venv



In [34]:
process_one = subprocess.run(['ls', '-ls'], capture_output=True, text=True)
#use stdout and returncode attribute
print('process_one stdout: {}'.format(process_one.stdout))
print('process_one status: {}'.format(process_one.returncode))

process_one stdout: total 40
 4 -rw-r--r-- 1 toanbui1991 toanbui1991    40 Apr 25 14:43 README.md
32 -rw-r--r-- 1 toanbui1991 toanbui1991 31196 Apr 25 17:03 item_36_subprocess.ipynb
 0 -rw-r--r-- 1 toanbui1991 toanbui1991     0 Apr 25 17:00 process_stdout.txt
 4 drwxr-xr-x 6 toanbui1991 toanbui1991  4096 Apr 25 14:50 venv

process_one status: 0


In [35]:
process_one = subprocess.run(['ls', '-ls', 'wrong folder'], capture_output=True, text=True)
#use stdout and returncode attribute
print('processone stderr: {}'.format(process_one.stderr))
print('process_one stdout: {}'.format(process_one.stdout))
#returncode different than 0 mean process fail or not complete
print('process_one status: {}'.format(process_one.returncode))

processone stderr: ls: cannot access 'wrong folder': No such file or directory

process_one stdout: 
process_one status: 2


In [31]:
#set stdout of a process to a file
with open('process_stdout.txt', 'w') as file:
    process_one = subprocess.run(['ls', 'la'], stdout=file, text=True)

ls: cannot access 'la': No such file or directory


In [28]:
subprocess.run('ls -la')

FileNotFoundError: [Errno 2] No such file or directory: 'ls -la'